In [2]:
#imports
from pathlib import Path
import json
from collections import defaultdict
import numpy as np
import time
import gensim
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
%matplotlib inline
import re, string

In [3]:
# load COCO metadata
filename = "data/captions_train2014.json"
with Path(filename).open() as f:
    coco_data = json.load(f)

In [8]:
class coco_values:
    def __init__(self, coco_data):
        #make variables for images and captions
        self.coco_annotations = coco_data['annotations']
        self.coco_images = coco_data['images']
        self.coco_img_ids = [i['id'] for i in self.coco_images]
        self.coco_cap_ids = [i['id'] for i in self.coco_annotations]
    def get_captions(self, image_id):
            #Find the caption IDs by returning all caption IDs associated with an image ID
        return [i['id'] for i in self.coco_annotations if i['image_id'] == image_id]
    def get_image_from_cap(self, caption_id):
        #Find the image ID by seeing if any image had a caption ID match
        return [i['image_id'] for i in self.coco_annotations if i['id'] == caption_id][0]
    def get_caption_text(self, caption_id):
        #Take the text from a given caption ID
        return [i['caption'] for i in self.coco_annotations if i['id'] == caption_id][0]
    def get_url(self, image_id):
        return [i['coco_url'] for i in self.coco_images if i['id'] == image_id]

In [11]:
x = coco_values(coco_data)

In [12]:
x.get_captions(318556)

[48, 126, 219, 255, 3555]

In [13]:
x.get_image_from_cap(48)

318556

In [14]:
x.get_caption_text(48)

'A very clean and well decorated empty bathroom'

In [16]:
#load glove dataset
path = r"data/glove.6B.200d.txt.w2v"
t0 = time.time()
glove = KeyedVectors.load_word2vec_format(path, binary=False)
t1 = time.time()
print("elapsed %ss" % (t1 - t0))

elapsed 97.68171954154968s


In [32]:
def string2vectors(given_str,IDF=None):
    if IDF is None:
        print("No IDF given, loading GloVe-200d")
        path = r"data/glove.6B.200d.txt.w2v"
        t0 = time.time()
        IDF = KeyedVectors.load_word2vec_format(path, binary=False)
        t1 = time.time()
        print("GloVe loaded in: %ss" % (t1-t0))
    punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
    given_str = punc_regex.sub('', given_str)
    str_list = given_str.lower().split()
    return [IDF[word] if word in IDF else np.zeroes((200,)) for word in str_list]

In [33]:
#string2vectors("My dog loves me",glove)

In [36]:
class ImageToWord:
    def __init__(self, context_words, d):
        """ Initializes all of the encoder and decoder layers in our model, setting them
        as attributes of the model.
        
        Parameters
        ----------
        context_words : int
            The number of context words included in our vocabulary
            
        d : int
            The dimensionality of our word embeddings
        """
        
        # STUDENT CODE:
        self.encode = dense(context_words, d, weight_initializer=glorot_normal, bias=False)
        self.decode = dense(d, context_words, weight_initializer=glorot_normal, bias=False)
        
    
    
    def __call__(self, x):
        ''' Passes data as input to our model, performing a "forward-pass".
        
        This allows us to conveniently initialize a model `m` and then send data through it
        to be classified by calling `m(x)`.
        
        Parameters
        ----------
        x : Union[numpy.ndarray, mygrad.Tensor], shape=(M, context_words)
            A batch of data consisting of M words from the context matrix,
                each tracking the number of co-occurences with `context_words` words.
                
        Returns
        -------
        mygrad.Tensor, shape=(M, context_words)
            The result of passing the data through borth the encoder and decoder.
        '''
        
        # STUDENT CODE:
        return self.decode(self.encode(x))
    
    
    @property
    def parameters(self):
        """ A convenience function for getting all the parameters of our model.
        
        This can be accessed as an attribute, via `model.parameters` 
        
        Returns
        -------
        Tuple[Tensor, ...]
            A tuple containing all of the learnable parameters for our model"""
        
        # STUDENT CODE:
        return self.encode.parameters + self.decode.parameters